In [1]:
import argparse
import time
import torch
from model import SingleViewto3D

In [2]:
# I cannot use arg parser in jupyter notebook. Create a class to store the arguments
class Args:
    def __init__(self):
        self.arch = 'resnet18'
        self.vis_freq = 1000
        self.batch_size = 1
        self.num_workers = 0
        self.type = 'vox'
        self.n_points = 5000
        self.w_chamfer = 1.0
        self.w_smooth = 0.1
        self.load_checkpoint = False
        self.device = 'cuda'
        self.load_feat = False
args = Args()


In [32]:
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.load_feat = False
args.type = 'point'
args.n_points = 1000

model = SingleViewto3D(args)

checkpoint = torch.load(f'checkpoint_{args.type}.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

c:\Users\amanc\anaconda3\envs\rob831\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\amanc\anaconda3\envs\rob831\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SingleViewto3D(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [48]:
# Load gt_image_0.png
import numpy as np
from PIL import Image
from torchvision import transforms

image = Image.open('gt_img_0.png')
image = transforms.ToTensor()(image)
image = image[:3]
image = image.unsqueeze(0)
image = image.permute(0, 2, 3, 1)
print(image.shape)

torch.Size([1, 137, 137, 3])


In [49]:
predicition = model(image, args)

In [50]:
print(predicition.shape)


torch.Size([1, 1000, 3])


In [51]:
print(predicition[0])

tensor([[ 0.2090,  0.0005,  0.0722],
        [ 0.1796,  0.0154,  0.2426],
        [-0.2061,  0.0226,  0.0820],
        ...,
        [ 0.3122, -0.1139, -0.2225],
        [-0.1948, -0.0122,  0.0918],
        [ 0.2089, -0.0018,  0.0730]], grad_fn=<SelectBackward0>)


In [52]:
# given prediction and ground truth, generate saliency map
import matplotlib.pyplot as plt
import torch.nn.functional as F

def saliency_map(model, image, prediction, target, device):
    image = image.to(device)
    image.requires_grad = True
    prediction = prediction.to(device)
    target = target.to(device)

    model.eval()
    model.zero_grad()
    output = model(image, args)
    loss = F.mse_loss(output, target)
    loss.backward()
    saliency = image.grad.data
    saliency = saliency.abs().max(dim=1)[0]
    saliency = saliency.squeeze()
    saliency = saliency - saliency.min()
    saliency = saliency / saliency.max()
    return saliency

saliency = saliency_map(model, image, predicition, image, args.device)
saliency = saliency.cpu().numpy()
plt.imshow(saliency, cmap='hot')

C:\Users\amanc\AppData\Local\Temp\ipykernel_40840\246368637.py:14: UserWarning: Using a target size (torch.Size([1, 137, 137, 3])) that is different to the input size (torch.Size([1, 1000, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output, target)


RuntimeError: The size of tensor a (1000) must match the size of tensor b (137) at non-singleton dimension 2